# Piper TTS Fine-tuning (Local Training)

This notebook guides you through fine-tuning the `en_US-lessac-medium` model on your **local machine**.

## Important Note on GPU Acceleration
**Your System**: Intel Mac (x86_64)  
**MPS Availability**: ❌ MPS (Metal Performance Shaders) GPU acceleration is **only available on Apple Silicon Macs** (M1/M2/M3), not Intel Macs.

**Training Options**:
1. **CPU Training** (This notebook) - Will work but slower
2. **Cloud GPU** - Use Google Colab with T4 GPU for faster training
3. **External GPU** - If you have an eGPU setup with CUDA

In [ ]:
import torch

# Check for GPU availability
if torch.cuda.is_available():
    print("✅ CUDA GPU Available (NVIDIA)")
    !nvidia-smi
elif torch.backends.mps.is_available():
    # Note: MPS may report as 'available' even on Intel Macs, but won't actually work
    import platform
    if platform.machine() == 'arm64':
        print("✅ Apple Silicon detected - MPS GPU available")
    else:
        
        print("ℹ️ Intel Mac detected - MPS requires Apple Silicon (M1/M2/M3)")
        print("Training will run on CPU")
else:
    print("⚠️ No GPU detected. Training will run on CPU (slower but will work).")

ℹ️ Intel Mac detected - MPS requires Apple Silicon (M1/M2/M3)
Training will run on CPU


### 1. Build Monotonic Align (Required)
We need to build the Cython extension `monotonic_align` which is required for VITS training.

In [ ]:
import os
import subprocess
import sys

# Ensure Cython/numpy is installed
print("Installing build dependencies...")
!{sys.executable} -m pip install -q cython numpy

# Define path to the monotonic_align directory
monotonic_align_path = os.path.abspath("../piper_repo/src/python/piper_train/vits/monotonic_align")

print(f"Building monotonic_align extension in: {monotonic_align_path}")

try:
    subprocess.check_call(
        f"cd '{monotonic_align_path}' && '{sys.executable}' setup.py build_ext --inplace", 
        shell=True,
        stdout=subprocess.DEVNULL,
        stderr=subprocess.PIPE
    )
    print("✅ Build complete.")
except subprocess.CalledProcessError as e:
    if os.path.exists(os.path.join(monotonic_align_path, 'core.cpython-311-darwin.so')):
        print("✅ Extension already built.")
    else:
        print("❌ Build failed! Error:", e.stderr.decode() if e.stderr else '')
        raise

Installing build dependencies...
Building monotonic_align extension in: /Users/rutwik/piper-model-training/piper_repo/src/python/piper_train/vits/monotonic_align
✅ Build complete.


### 2. Setup Environment Variables

In [ ]:
import os

piper_src_path = os.path.abspath("../piper_repo/src/python")
print(f"Piper Source Path: {piper_src_path}")

if not os.path.exists(piper_src_path):
    print("⚠️ WARNING: piper repository path not found! Check your directories.")

Piper Source Path: /Users/rutwik/piper-model-training/piper_repo/src/python


### 3. Download Pre-trained Checkpoint

In [ ]:
import urllib.request
from pathlib import Path

os.makedirs("checkpoints", exist_ok=True)

checkpoint_url = "https://huggingface.co/datasets/rhasspy/piper-checkpoints/resolve/main/en/en_US/lessac/medium/epoch%3D2164-step%3D1355540.ckpt"
checkpoint_path = "checkpoints/epoch=2164-step=1355540.ckpt"

if not os.path.exists(checkpoint_path):
    print("📥 Downloading checkpoint (this may take a few minutes)...")
    urllib.request.urlretrieve(checkpoint_url, checkpoint_path)
    print(f"✅ Checkpoint downloaded to: {checkpoint_path}")
else:
    print(f"✅ Checkpoint already exists at: {checkpoint_path}")

✅ Checkpoint already exists at: checkpoints/epoch=2164-step=1355540.ckpt


### 4. Monitor Training with TensorBoard
Start TensorBoard to visualize training progress.

In [ ]:
%load_ext tensorboard
%tensorboard --logdir training_dir/lightning_logs

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 10653), started 0:33:20 ago. (Use '!kill 10653' to kill it.)

### 5. Start Fine-tuning

**Configuration**:
- `--accelerator`: Automatically detects available accelerator (CPU/GPU)
- `--batch-size 4`: Small batch for memory efficiency
- `--max_epochs 10000`: Will train until stopped or complete

**Performance Note**: CPU training is significantly slower than GPU. Consider:
- Reducing `--max_epochs` for testing
- Using Google Colab with free T4 GPU
- Training overnight with current settings

In [ ]:
import torch
import sys
import platform

# Intelligently determine accelerator
if torch.cuda.is_available():
    accelerator = "gpu"
    print("🚀 Using: CUDA GPU")
elif torch.backends.mps.is_available() and platform.machine() == 'arm64':
    accelerator = "mps"
    print("🚀 Using: Apple Silicon GPU (MPS)")
else:
    accelerator = "cpu"
    print("🐌 Using: CPU (This will be slow - consider using Google Colab for GPU training)")

command = f"""
PYTHONPATH=\"{piper_src_path}\" \"{sys.executable}\" -m piper_train \
  --dataset-dir training_dir \
  --accelerator '{accelerator}' \
  --devices 1 \
  --batch-size 4 \
  --validation-split 0.0 \
  --num-test-examples 0 \
  --max_epochs 10000 \
  --resume_from_checkpoint \"{checkpoint_path}\" \
  --checkpoint-epochs 1 \
  --quality medium \
  --precision 32
"""

print("\n" + "="*60)
print("Starting training...")
print("You can monitor progress in TensorBoard above")
print("Press 'Stop' button to interrupt training")
print("="*60 + "\n")

!{command}

🐌 Using: CPU (This will be slow - consider using Google Colab for GPU training)

Starting training...
You can monitor progress in TensorBoard above
Press 'Stop' button to interrupt training

DEBUG:piper_train:Namespace(dataset_dir='training_dir', checkpoint_epochs=1, quality='medium', resume_from_single_speaker_checkpoint=None, max_epochs=10000, accelerator='cpu', devices='1', precision='32', default_root_dir=None, resume_from_checkpoint='checkpoints/epoch=2164-step=1355540.ckpt', batch_size=4, validation_split=0.0, num_test_examples=0, max_phoneme_ids=None, hidden_channels=192, inter_channels=192, filter_channels=768, n_layers=6, n_heads=2, seed=1234)
DEBUG:piper_train:Checkpoints will be saved every 1 epoch(s)
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
/Users/rutwik/miniconda3/envs/voice_training/lib/python3.11/site-packages/torch/nn/utils/weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizatio